In [1]:
import os
import sys
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from scipy import stats
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'
import matplotlib.dates as mdates

def get_hstr(path):
    hstr = pd.read_csv(path)
    hstr['datetime'] = pd.to_datetime(hstr['CLCT_UNIX_TM'], unit='s', utc=True)
    hstr['datetime'] = hstr['datetime'].dt.tz_convert('Asia/Seoul').dt.strftime('%Y-%m-%d %H:%M:%S')
    hstr['datetime'] = hstr['datetime'].astype('datetime64[ns]')
    hstr.columns = ['현장교차로ID', '수집유닉스시각', '수집일시', '제어구분코드', '제어상태코드', '주기시간', '옵셋시간',
        'A링1현시시간', 'A링2현시시간', 'A링3현시시간', 'A링4현시시간', 'A링5현시시간', 'A링6현시시간',
        'A링7현시시간', 'A링8현시시간', 'B링1현시시간', 'B링2현시시간', 'B링3현시시간', 'B링4현시시간',
        'B링5현시시간', 'B링6현시시간', 'B링7현시시간', 'B링8현시시간', 'A링1현시보행시간',
        'A링2현시보행시간', 'A링3현시보행시간', 'A링4현시보행시간', 'A링5현시보행시간', 'A링6현시보행시간',
        'A링7현시보행시간', 'A링8현시보행시간', 'B링1현시보행시간', 'B링2현시보행시간', 'B링3현시보행시간',
        'B링4현시보행시간', 'B링5현시보행시간', 'B링6현시보행시간', 'B링7현시보행시간', 'B링8현시보행시간',
        '수집날짜시각']
    cols = list(hstr.columns).copy()
    cols.remove('수집날짜시각')
    cols.insert(1, '수집날짜시각')
    hstr = hstr[cols]
    return hstr

inter_nos = list(range(5031, 5048))

In [2]:
# 성남시 신호이력 데이터
s_tod_his = pd.read_csv('seongnam/S_TOD_HIS_1702597659892.csv')

# 인천시 신호이력 데이터
'''SELECT * FROM soitdspotintsoperhstr WHERE SPOT_INTS_ID BETWEEN 5031 AND 5047'''
csv_files = [f for f in os.listdir('incheon') if f.endswith('.csv')]

dfs = []
for file in csv_files:
    df_name = file.replace('.csv', '')
    print(df_name)
    globals()[df_name] = get_hstr(f'incheon/{file}')
    dfs.append(globals()[df_name])
    print(sorted([dt for dt in globals()[df_name]['수집날짜시각']])[0])
    print(sorted([dt for dt in globals()[df_name]['수집날짜시각']])[-1])
dfs = [df[(df.수집날짜시각.dt.hour!=23)&df.수집날짜시각.dt.minute!=59] for df in dfs]
hstr = pd.concat(dfs).drop_duplicates()
hstr = hstr[hstr.수집날짜시각.dt.day >= 12]
hstr = hstr[hstr.현장교차로ID.isin(inter_nos)].sort_values(by='수집날짜시각').reset_index(drop=True)
hstr = hstr[['현장교차로ID', '수집날짜시각', '제어구분코드', '제어상태코드', '주기시간', '옵셋시간',
             'A링1현시시간', 'A링2현시시간', 'A링3현시시간', 'A링4현시시간',
             'A링5현시시간', 'A링6현시시간', 'A링7현시시간', 'A링8현시시간',
             'B링1현시시간', 'B링2현시시간', 'B링3현시시간', 'B링4현시시간',
             'B링5현시시간', 'B링6현시시간', 'B링7현시시간', 'B링8현시시간']]
print(len(hstr))
print(sorted([dt for dt in hstr['수집날짜시각'] if dt.hour != 23])[0])
print(sorted([dt for dt in hstr['수집날짜시각'] if dt.hour != 23])[-1])
hstr = hstr.sort_values(by=['현장교차로ID', '수집날짜시각']).reset_index(drop=True)

soitdspotintsoperhstr_202312110921
2023-12-10 09:00:00
2023-12-11 23:59:59
soitdspotintsoperhstr_202312120814
2023-12-11 09:00:57
2023-12-12 23:59:55
soitdspotintsoperhstr_202312131102
2023-12-12 09:00:58
2023-12-13 23:59:56
soitdspotintsoperhstr_202312141254
2023-12-13 09:00:58
2023-12-14 23:59:59
soitdspotintsoperhstr_202312150926
2023-12-14 09:00:58
2023-12-15 23:59:30
soitdspotintsoperhstr_202312151700
2023-12-15 09:00:58
2023-12-15 23:59:30
32289
2023-12-12 00:00:09
2023-12-15 16:59:54


In [34]:
# 제어구분코드 (0: 비감응 지역제어, 1: 감응 지역제어, 2: 감응 중앙제어, 3: 비감응 중앙제어)
# 제어상태코드 (0:오프라인, 1:온라인, 2:트랜스, 3:통신장애, 4:수동)
print(hstr.제어구분코드.unique())
print(hstr.제어상태코드.unique())
print(hstr.제어구분코드.value_counts())
print(hstr.제어상태코드.value_counts())
S0 = set(hstr[hstr.제어구분코드==0].index)
S3 = set(hstr[hstr.제어구분코드==3].index)
T0 = set(hstr[hstr.제어구분코드==0].index)
T1 = set(hstr[hstr.제어구분코드==1].index)
T2 = set(hstr[hstr.제어구분코드==2].index)
T3 = set(hstr[hstr.제어구분코드==3].index)
k=6
S0_indices = list(hstr[hstr.제어구분코드==0].index)
for j in range(10):
    i = S0_indices[j]
    context = range(i - k, i + k - 1)
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(hstr.iloc[context])

,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,A링5현시시간,A링6현시시간,A링7현시시간,A링8현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,B링7현시시간,B링8현시시간
773,5031,2023-12-13 10:23:40,3,1,160,140,49,20,27,29,35,0,0,0,49,20,27,29,35,0,0,0
774,5031,2023-12-13 10:26:20,3,1,160,140,49,20,27,29,35,0,0,0,49,20,27,29,35,0,0,0
775,5031,2023-12-13 10:29:00,3,1,160,140,49,20,27,29,35,0,0,0,49,20,27,29,35,0,0,0
776,5031,2023-12-13 10:31:40,3,1,160,140,49,20,27,29,35,0,0,0,49,20,27,29,35,0,0,0
777,5031,2023-12-13 10:34:20,3,1,160,140,49,20,27,29,35,0,0,0,49,20,27,29,35,0,0,0
778,5031,2023-12-13 10:39:07,3,1,31,107,49,20,27,156,35,0,0,0,49,20,27,156,35,0,0,0
779,5031,2023-12-13 10:41:27,0,0,140,87,37,16,27,25,35,0,0,0,37,16,27,25,35,0,0,0
780,5031,2023-12-13 10:45:00,3,2,213,140,69,31,31,43,39,0,0,0,69,31,31,43,39,0,0,0
781,5031,2023-12-13 10:47:40,3,1,160,140,49,20,27,29,35,0,0,0,49,20,27,29,35,0,0,0
782,5031,2023-12-13 10:50:20,3,1,160,140,49,20,27,29,35,0,0,0,49,20,27,29,35,0,0,0


,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,A링5현시시간,A링6현시시간,A링7현시시간,A링8현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,B링7현시시간,B링8현시시간
907,5031,2023-12-13 17:53:38,3,1,180,159,65,20,27,33,35,0,0,0,65,20,27,33,35,0,0,0
908,5031,2023-12-13 17:56:38,3,1,180,159,65,20,27,33,35,0,0,0,65,20,27,33,35,0,0,0
909,5031,2023-12-13 17:59:38,3,1,180,159,65,20,27,33,35,0,0,0,65,20,27,33,35,0,0,0
910,5031,2023-12-13 18:02:38,3,1,180,159,65,20,27,33,35,0,0,0,65,20,27,33,35,0,0,0
911,5031,2023-12-13 18:05:38,3,1,180,159,65,20,27,33,35,0,0,0,65,20,27,33,35,0,0,0
912,5031,2023-12-13 18:10:47,3,1,52,107,65,20,27,161,35,0,0,0,65,20,27,161,35,0,0,0
913,5031,2023-12-13 18:13:07,0,0,140,67,37,16,27,25,35,0,0,0,37,16,27,25,35,0,0,0
914,5031,2023-12-13 18:17:06,3,2,239,126,90,31,31,48,39,0,0,0,90,31,31,48,39,0,0,0
915,5031,2023-12-13 18:20:39,3,2,213,159,78,25,31,40,39,0,0,0,78,25,31,40,39,0,0,0
916,5031,2023-12-13 18:23:39,3,1,180,160,65,20,27,33,35,0,0,0,65,20,27,33,35,0,0,0


,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,A링5현시시간,A링6현시시간,A링7현시시간,A링8현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,B링7현시시간,B링8현시시간
1028,5031,2023-12-13 23:38:28,3,2,186,128,54,26,31,36,39,0,0,0,54,26,31,36,39,0,0,0
1029,5031,2023-12-13 23:41:00,3,2,152,0,41,17,29,27,38,0,0,0,41,17,29,27,38,0,0,0
1030,5031,2023-12-13 23:43:20,3,1,140,0,37,16,27,25,35,0,0,0,37,16,27,25,35,0,0,0
1031,5031,2023-12-13 23:45:40,3,1,140,0,37,16,27,25,35,0,0,0,37,16,27,25,35,0,0,0
1032,5031,2023-12-13 23:48:00,3,1,140,0,37,16,27,25,35,0,0,0,37,16,27,25,35,0,0,0
1033,5031,2023-12-13 23:52:43,3,1,26,3,179,16,27,25,35,0,0,0,179,16,27,25,35,0,0,0
1034,5031,2023-12-13 23:55:02,0,0,140,2,37,16,27,25,35,0,0,0,37,16,27,25,35,0,0,0
1035,5031,2023-12-13 23:57:22,3,1,140,2,37,16,27,25,35,0,0,0,37,16,27,25,35,0,0,0
1036,5031,2023-12-13 23:59:43,3,1,140,2,37,16,27,25,35,0,0,0,37,16,27,25,35,0,0,0
1037,5031,2023-12-14 00:02:03,3,1,140,122,37,16,27,25,35,0,0,0,37,16,27,25,35,0,0,0


,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,A링5현시시간,A링6현시시간,A링7현시시간,A링8현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,B링7현시시간,B링8현시시간
1757,5031,2023-12-15 07:01:11,3,1,160,152,43,25,27,30,35,0,0,0,43,25,27,30,35,0,0,0
1758,5031,2023-12-15 07:03:52,3,1,160,52,43,25,27,30,35,0,0,0,43,25,27,30,35,0,0,0
1759,5031,2023-12-15 07:07:50,3,2,239,111,70,51,31,48,39,0,0,0,70,51,31,48,39,0,0,0
1760,5031,2023-12-15 07:11:40,3,2,230,161,66,48,31,46,39,0,0,0,66,48,31,46,39,0,0,0
1761,5031,2023-12-15 07:14:41,3,1,180,161,49,35,27,34,35,0,0,0,49,35,27,34,35,0,0,0
1762,5031,2023-12-15 07:17:41,3,1,180,161,49,35,27,34,35,0,0,0,49,35,27,34,35,0,0,0
1763,5031,2023-12-15 07:21:58,0,0,1,58,126,35,27,34,35,0,0,0,126,35,27,34,35,0,0,0
1764,5031,2023-12-15 07:24:18,0,0,140,19,37,16,27,25,35,0,0,0,37,16,27,25,35,0,0,0
1765,5031,2023-12-15 07:26:49,3,2,151,169,39,25,27,25,35,0,0,0,39,25,27,25,35,0,0,0
1766,5031,2023-12-15 07:29:41,3,2,172,161,47,32,27,31,35,0,0,0,47,32,27,31,35,0,0,0


,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,A링5현시시간,A링6현시시간,A링7현시시간,A링8현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,B링7현시시간,B링8현시시간
1758,5031,2023-12-15 07:03:52,3,1,160,52,43,25,27,30,35,0,0,0,43,25,27,30,35,0,0,0
1759,5031,2023-12-15 07:07:50,3,2,239,111,70,51,31,48,39,0,0,0,70,51,31,48,39,0,0,0
1760,5031,2023-12-15 07:11:40,3,2,230,161,66,48,31,46,39,0,0,0,66,48,31,46,39,0,0,0
1761,5031,2023-12-15 07:14:41,3,1,180,161,49,35,27,34,35,0,0,0,49,35,27,34,35,0,0,0
1762,5031,2023-12-15 07:17:41,3,1,180,161,49,35,27,34,35,0,0,0,49,35,27,34,35,0,0,0
1763,5031,2023-12-15 07:21:58,0,0,1,58,126,35,27,34,35,0,0,0,126,35,27,34,35,0,0,0
1764,5031,2023-12-15 07:24:18,0,0,140,19,37,16,27,25,35,0,0,0,37,16,27,25,35,0,0,0
1765,5031,2023-12-15 07:26:49,3,2,151,169,39,25,27,25,35,0,0,0,39,25,27,25,35,0,0,0
1766,5031,2023-12-15 07:29:41,3,2,172,161,47,32,27,31,35,0,0,0,47,32,27,31,35,0,0,0
1767,5031,2023-12-15 07:32:41,3,1,180,161,49,35,27,34,35,0,0,0,49,35,27,34,35,0,0,0


,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,A링5현시시간,A링6현시시간,A링7현시시간,A링8현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,B링7현시시간,B링8현시시간
1950,5031,2023-12-15 15:54:21,3,1,160,140,49,20,27,29,35,0,0,0,49,20,27,29,35,0,0,0
1951,5031,2023-12-15 15:57:01,3,1,160,140,49,20,27,29,35,0,0,0,49,20,27,29,35,0,0,0
1952,5031,2023-12-15 15:59:41,3,1,160,140,49,20,27,29,35,0,0,0,49,20,27,29,35,0,0,0
1953,5031,2023-12-15 16:02:21,3,1,160,140,49,20,27,29,35,0,0,0,49,20,27,29,35,0,0,0
1954,5031,2023-12-15 16:05:01,3,1,160,100,49,20,27,29,35,0,0,0,49,20,27,29,35,0,0,0
1955,5031,2023-12-15 16:08:44,3,2,223,154,80,29,31,44,39,0,0,0,80,29,31,44,39,0,0,0
1956,5031,2023-12-15 16:11:33,0,0,169,153,58,19,27,30,35,0,0,0,58,19,27,30,35,0,0,0
1957,5031,2023-12-15 16:14:22,0,0,170,153,42,23,31,35,39,0,0,0,42,23,31,35,39,0,0,0
1958,5031,2023-12-15 16:17:12,3,1,170,153,58,19,27,31,35,0,0,0,58,19,27,31,35,0,0,0
1959,5031,2023-12-15 16:20:02,3,1,170,153,58,19,27,31,35,0,0,0,58,19,27,31,35,0,0,0


,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,A링5현시시간,A링6현시시간,A링7현시시간,A링8현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,B링7현시시간,B링8현시시간
1951,5031,2023-12-15 15:57:01,3,1,160,140,49,20,27,29,35,0,0,0,49,20,27,29,35,0,0,0
1952,5031,2023-12-15 15:59:41,3,1,160,140,49,20,27,29,35,0,0,0,49,20,27,29,35,0,0,0
1953,5031,2023-12-15 16:02:21,3,1,160,140,49,20,27,29,35,0,0,0,49,20,27,29,35,0,0,0
1954,5031,2023-12-15 16:05:01,3,1,160,100,49,20,27,29,35,0,0,0,49,20,27,29,35,0,0,0
1955,5031,2023-12-15 16:08:44,3,2,223,154,80,29,31,44,39,0,0,0,80,29,31,44,39,0,0,0
1956,5031,2023-12-15 16:11:33,0,0,169,153,58,19,27,30,35,0,0,0,58,19,27,30,35,0,0,0
1957,5031,2023-12-15 16:14:22,0,0,170,153,42,23,31,35,39,0,0,0,42,23,31,35,39,0,0,0
1958,5031,2023-12-15 16:17:12,3,1,170,153,58,19,27,31,35,0,0,0,58,19,27,31,35,0,0,0
1959,5031,2023-12-15 16:20:02,3,1,170,153,58,19,27,31,35,0,0,0,58,19,27,31,35,0,0,0
1960,5031,2023-12-15 16:22:52,3,1,170,153,58,19,27,31,35,0,0,0,58,19,27,31,35,0,0,0


,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,A링5현시시간,A링6현시시간,A링7현시시간,A링8현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,B링7현시시간,B링8현시시간
2161,5032,2023-12-12 07:28:35,3,1,170,55,24,61,24,37,24,0,0,0,24,61,24,37,24,0,0,0
2162,5032,2023-12-12 07:31:25,3,1,170,55,24,61,24,37,24,0,0,0,24,61,24,37,24,0,0,0
2163,5032,2023-12-12 07:34:15,3,1,170,55,24,61,24,37,24,0,0,0,24,61,24,37,24,0,0,0
2164,5032,2023-12-12 07:37:05,3,1,170,55,24,61,24,37,24,0,0,0,24,61,24,37,24,0,0,0
2165,5032,2023-12-12 07:39:55,3,1,170,55,24,61,24,37,24,0,0,0,24,61,24,37,24,0,0,0
2166,5032,2023-12-12 07:42:27,3,1,133,37,0,55,21,37,20,0,0,0,0,55,21,37,20,0,0,0
2167,5032,2023-12-12 07:44:24,0,0,117,154,18,24,19,37,19,0,0,0,18,24,19,37,19,0,0,0
2168,5032,2023-12-12 07:48:10,3,2,226,40,35,84,35,37,35,0,0,0,35,84,35,37,35,0,0,0
2169,5032,2023-12-12 07:51:15,3,2,185,55,27,67,27,37,27,0,0,0,27,67,27,37,27,0,0,0
2170,5032,2023-12-12 07:54:05,3,1,170,55,24,61,24,37,24,0,0,0,24,61,24,37,24,0,0,0


,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,A링5현시시간,A링6현시시간,A링7현시시간,A링8현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,B링7현시시간,B링8현시시간
2625,5032,2023-12-13 05:02:01,3,1,140,60,20,43,20,37,20,0,0,0,20,43,20,37,20,0,0,0
2626,5032,2023-12-13 05:04:21,3,1,140,60,20,43,20,37,20,0,0,0,20,43,20,37,20,0,0,0
2627,5032,2023-12-13 05:06:41,3,1,140,60,20,43,20,37,20,0,0,0,20,43,20,37,20,0,0,0
2628,5032,2023-12-13 05:09:01,3,1,140,61,20,43,20,37,20,0,0,0,20,43,20,37,20,0,0,0
2629,5032,2023-12-13 05:11:20,3,1,139,60,20,43,20,37,19,0,0,0,20,43,20,37,19,0,0,0
2630,5032,2023-12-13 05:13:40,3,1,141,60,20,43,20,38,20,0,0,0,20,43,20,38,20,0,0,0
2631,5032,2023-12-13 05:17:53,0,0,252,32,132,43,20,37,20,0,0,0,132,43,20,37,20,0,0,0
2632,5032,2023-12-13 05:19:50,0,0,117,9,18,24,19,37,19,0,0,0,18,24,19,37,19,0,0,0
2633,5032,2023-12-13 05:22:55,3,2,186,55,30,60,30,37,29,0,0,0,30,60,30,37,29,0,0,0
2634,5032,2023-12-13 05:25:20,3,2,145,60,21,45,21,37,21,0,0,0,21,45,21,37,21,0,0,0


,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,A링5현시시간,A링6현시시간,A링7현시시간,A링8현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,B링7현시시간,B링8현시시간
2626,5032,2023-12-13 05:04:21,3,1,140,60,20,43,20,37,20,0,0,0,20,43,20,37,20,0,0,0
2627,5032,2023-12-13 05:06:41,3,1,140,60,20,43,20,37,20,0,0,0,20,43,20,37,20,0,0,0
2628,5032,2023-12-13 05:09:01,3,1,140,61,20,43,20,37,20,0,0,0,20,43,20,37,20,0,0,0
2629,5032,2023-12-13 05:11:20,3,1,139,60,20,43,20,37,19,0,0,0,20,43,20,37,19,0,0,0
2630,5032,2023-12-13 05:13:40,3,1,141,60,20,43,20,38,20,0,0,0,20,43,20,38,20,0,0,0
2631,5032,2023-12-13 05:17:53,0,0,252,32,132,43,20,37,20,0,0,0,132,43,20,37,20,0,0,0
2632,5032,2023-12-13 05:19:50,0,0,117,9,18,24,19,37,19,0,0,0,18,24,19,37,19,0,0,0
2633,5032,2023-12-13 05:22:55,3,2,186,55,30,60,30,37,29,0,0,0,30,60,30,37,29,0,0,0
2634,5032,2023-12-13 05:25:20,3,2,145,60,21,45,21,37,21,0,0,0,21,45,21,37,21,0,0,0
2635,5032,2023-12-13 05:27:40,3,1,140,60,20,43,20,37,20,0,0,0,20,43,20,37,20,0,0,0
